## 5ª Rodada de Questões do CEO 

### Bibliotecas

In [1]:
from sqlalchemy import create_engine
import pandas as pd

### Acessando o Banco de Dados


In [2]:
# Conexão com o Banco
engine = create_engine('sqlite:///db_olist.sqlite')

# Criando a conexão com o Banco
cursor = engine.connect()

**1.** Qual o número de clientes únicos nos estado de Minas Gerais ou Rio de Janeiro?


In [3]:
query = """
    SELECT
        c.customer_state, 
        COUNT(DISTINCT c.customer_id) AS num_unique_customers
    FROM 
        customer c 
    WHERE
        c.customer_state = 'MG' OR c.customer_state = 'SP'
    GROUP BY 	
        c.customer_state 
        """
pd.read_sql(query, cursor)

,customer_state,num_unique_customers
0,MG,11635
1,SP,41746


**2.** Qual a quantidade de cidades únicas dos vendedores no estado de São Paulo ou Rio de Janeiro com a latitude maior que -24.54 e longitude menor que -45.63?


In [4]:
query = """
    SELECT
        g.geolocation_state, 
        COUNT(DISTINCT g.geolocation_city) AS quant_unique_cities 
    FROM
        geolocation g 
    WHERE
        g.geolocation_state = 'SP' OR g.geolocation_state = 'RJ'
        AND (g.geolocation_lat  > -24.54 AND g.geolocation_lng < -45.63)
    GROUP BY 
        g.geolocation_state 
        """

pd.read_sql(query, cursor)

,geolocation_state,quant_unique_cities
0,RJ,9
1,SP,1048


**3.** Qual o número total de pedidos únicos, o número total de produtos e o preço médio dos pedidos com o preço de frete maior que  20,00 e a data limite de envio entre os dias 1 e 31 de Outubro de 2016?


In [5]:
query = """
          SELECT
        COUNT(DISTINCT oi.order_id)   AS total_orders,
        COUNT(DISTINCT oi.product_id) AS total_products,
        AVG(oi.price)                 AS mean_price 
    FROM 
        order_items oi 
    WHERE
        oi.freight_value > 20.00
        AND (DATE(oi.shipping_limit_date) >= '2016-10-01'AND DATE(oi.shipping_limit_date) <= '2016-10-31')
        """

pd.read_sql(query, cursor)

,total_orders,total_products,mean_price
0,103,91,229.672308




**4.** Mostre a quantidade total dos pedidos e o valor total do pagamento, para pagamentos entre 1 e 5 prestações ou um valor de pagamento acima de 5.000,00
    


In [6]:
query = """
      SELECT 
        op.payment_type, op.payment_installments, 
        COUNT(op.order_id) AS total_orders,
        SUM(op.payment_value) AS total_payments_value 
    FROM 
        order_payments op 
    WHERE
       (op.payment_installments >= 1 AND op.payment_installments  <= 5) 
       OR op.payment_value > 5000
    GROUP BY 
        op.payment_type, op.payment_installments 
        """

pd.read_sql(query, cursor)

,payment_type,payment_installments,total_orders,total_payments_value
0,boleto,1,19784,2869361.27
1,credit_card,1,25455,2440445.43
2,credit_card,2,12413,1579283.03
3,credit_card,3,10461,1491103.80
4,credit_card,4,7098,1163907.61
5,credit_card,5,5239,961174.30
6,credit_card,8,1,6929.31
7,debit_card,1,1529,217989.79
8,not_defined,1,3,0.00
9,voucher,1,5775,379436.87


**5.** Qual a quantidade de pedidos com o status em processamento ou cancelada acontecem com a data estimada de entrega maior que 01 de Janeiro de 2017 ou menor que 23 de Novembro de 2016?



In [7]:
query = """
    SELECT 
        o.order_status, 
        COUNT(o.order_status) AS num_orders 
    FROM
        orders o 
    WHERE
        (o.order_status = 'canceled' OR o.order_status = 'processing') AND 
        (DATE(o.order_estimated_delivery_date) > '2017-01-01' OR DATE(o.order_estimated_delivery_date) < '2016-11-23')
    GROUP BY 
        o.order_status 
        """

pd.read_sql(query, cursor)

,order_status,num_orders
0,canceled,611
1,processing,299


**6.** Quantos produtos estão cadastrados nas categorias: perfumaria, brinquedos, esporte lazer, cama mesa e banho e móveis de escritório que possuem mais de 5 fotos, um peso maior que 5 g, um altura maior que 10 cm, uma largura maior que 20 cm?

In [8]:
query = """
    SELECT
        p.product_category_name, 
        COUNT(DISTINCT p.product_id)  AS num_products 
    FROM
        products p 
    WHERE
        ( p.product_category_name = 'perfumaria'       
         OR p.product_category_name = 'brinquedos'       
         OR p.product_category_name = 'esporte_lazer'    
         OR p.product_category_name = 'cama_mesa_banho' 
         OR p.product_category_name = 'moveis_escritorio' ) 
          AND (p.product_photos_qty > 5) 
          AND (p.product_weight_g > 5) 
          AND (p.product_height_cm > 10) 
          AND (p.product_width_cm > 20)
    GROUP BY 
        p.product_category_name 
        """

pd.read_sql(query, cursor)

,product_category_name,num_products
0,brinquedos,30
1,cama_mesa_banho,22
2,esporte_lazer,69
3,moveis_escritorio,7
4,perfumaria,6
